In [1]:
import pandas as pd
import numpy as np
import os, sys, glob
from tqdm import tqdm
import datetime as dt

In [2]:
df_event = pd.read_csv('/Users/graceli/Documents/Baruch MSBA/S3/BUS9430/Data/FinalData/Events.csv')

In [3]:
df_event = df_event.drop(['Event ID', 'Event Borough','Police Precinct'], axis=1)

In [4]:
df_event.shape

(188481, 4)

In [5]:
df_event.head()

,Start Date/Time,End Date/Time,Event Type,Zipcode
0,2017-01-01 00:00:00,2017-01-01 23:59:00,Construction,10075
1,2017-01-01 00:00:00,2017-01-01 23:59:00,Construction,10075
2,2017-01-01 00:00:00,2017-01-01 12:00:00,Construction,10010
3,2017-01-01 00:00:00,2017-01-01 01:00:00,Construction,10075
4,2017-01-01 00:00:00,2017-01-01 12:00:00,Construction,10010


In [6]:
# df_event = df_event.drop_duplicates()
df_event.shape
# df_event['dum'] = 1
# df_event = df_event.groupby(['Start Date/Time','End Date/Time','Zipcode','Event Type']).agg({'dum':['count']}).reset_index()
# df_event.columns = df_event.columns.droplevel(1)
# df_event = df_event.rename(columns={'dum':'Event Count'})
# df_event.head()



(188481, 4)

In [7]:
df_event_clean = df_event.loc[df_event['Start Date/Time']<df_event['End Date/Time'],:]
df_event_clean.head()

,Start Date/Time,End Date/Time,Event Type,Zipcode
0,2017-01-01 00:00:00,2017-01-01 23:59:00,Construction,10075
1,2017-01-01 00:00:00,2017-01-01 23:59:00,Construction,10075
2,2017-01-01 00:00:00,2017-01-01 12:00:00,Construction,10010
3,2017-01-01 00:00:00,2017-01-01 01:00:00,Construction,10075
4,2017-01-01 00:00:00,2017-01-01 12:00:00,Construction,10010


In [9]:
allEvents = df_event_clean['Event Type'].value_counts()
outEvents = allEvents[allEvents > 99].index.values

allTime = np.arange(pd.to_datetime('20170101 00:00:00'), pd.to_datetime('20171231 23:59:00'),
                    pd.to_datetime('20170101 01:00:00') - pd.to_datetime('20170101 00:00:00'))

def perZipcode(dfSub):
    dfOut = pd.DataFrame({'time' : allTime})
    dfOut['time'] = dfOut['time'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
    for event in outEvents:
        dfOut[event] = 0
    
    for row in dfSub.to_records():
        if row['Event Type'] in outEvents:
            dfOut.loc[(dfOut.time >= row['Start Date/Time']) & 
                      (dfOut.time < row['End Date/Time']),
                      row['Event Type']] += 1
    print(dfSub.Zipcode.iloc[0], 'finished')
    return dfOut

dfOut = df_event_clean.groupby('Zipcode').apply(perZipcode).reset_index()
dfOut.drop('level_1', axis=1, inplace=True)

10000 finished
10000 finished
10001 finished
10002 finished
10003 finished
10004 finished
10005 finished
10006 finished
10007 finished
10009 finished
10010 finished
10011 finished
10012 finished
10013 finished
10014 finished
10016 finished
10017 finished
10018 finished
10019 finished
10021 finished
10022 finished
10023 finished
10024 finished
10025 finished
10026 finished
10027 finished
10029 finished
10030 finished
10031 finished
10032 finished
10033 finished
10034 finished
10035 finished
10036 finished
10037 finished
10038 finished
10039 finished
10040 finished
10044 finished
10065 finished
10069 finished
10075 finished
10107 finished
10111 finished
10115 finished
10128 finished
10153 finished
10155 finished
10158 finished
10162 finished
10199 finished
10275 finished
10278 finished
10280 finished
10281 finished
10282 finished
10301 finished
10302 finished
10303 finished
10304 finished
10305 finished
10306 finished
10307 finished
10308 finished
10309 finished
10310 finished
10312 fini

In [10]:
for col in outEvents:
    dfOut[col + '_01'] = (dfOut[col] > 0).astype(int)

In [11]:
dfOut.head()

,Zipcode,time,Special Event,Construction,Farmers Market,Sidewalk Sale,Parade,Production Event,Street Event,Plaza Event,...,Religious Event_01,Theater Load in and Load Outs_01,Miscellaneous_01,Embargo_01,Street Festival_01,Athletic Race / Tour_01,Single Block Festival_01,Stickball_01,Weekend Walk_01,Shooting Permit_01
0,10000,2017-01-01 00:00:00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10000,2017-01-01 01:00:00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10000,2017-01-01 02:00:00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10000,2017-01-01 03:00:00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10000,2017-01-01 04:00:00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
dfOut['hour'] = dfOut['time'].str.slice(11,13).astype(int)

In [13]:
# dfOut['time'] = dfOut['time'].str.slice(0,10)
dfOut['time'] = pd.to_datetime(dfOut['time'],format="%Y-%m-%d")
dfOut.head()

,Zipcode,time,Special Event,Construction,Farmers Market,Sidewalk Sale,Parade,Production Event,Street Event,Plaza Event,...,Theater Load in and Load Outs_01,Miscellaneous_01,Embargo_01,Street Festival_01,Athletic Race / Tour_01,Single Block Festival_01,Stickball_01,Weekend Walk_01,Shooting Permit_01,hour
0,10000,2017-01-01 00:00:00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10000,2017-01-01 01:00:00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,10000,2017-01-01 02:00:00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
3,10000,2017-01-01 03:00:00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
4,10000,2017-01-01 04:00:00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4


In [14]:
dfOut.columns

Index(['Zipcode', 'time', 'Special Event', 'Construction', 'Farmers Market',
       'Sidewalk Sale', 'Parade', 'Production Event', 'Street Event',
       'Plaza Event', 'Block Party', 'Religious Event',
       'Theater Load in and Load Outs', 'Miscellaneous', 'Embargo',
       'Street Festival', 'Athletic Race / Tour', 'Single Block Festival',
       'Stickball', 'Weekend Walk', 'Shooting Permit', 'Special Event_01',
       'Construction_01', 'Farmers Market_01', 'Sidewalk Sale_01', 'Parade_01',
       'Production Event_01', 'Street Event_01', 'Plaza Event_01',
       'Block Party_01', 'Religious Event_01',
       'Theater Load in and Load Outs_01', 'Miscellaneous_01', 'Embargo_01',
       'Street Festival_01', 'Athletic Race / Tour_01',
       'Single Block Festival_01', 'Stickball_01', 'Weekend Walk_01',
       'Shooting Permit_01', 'hour'],
      dtype='object')

In [15]:
dfOut.to_csv("/Users/graceli/Documents/Baruch MSBA/S3/BUS9430/Data/FinalData/Events_updated_v2.csv")

In [16]:
dfOut.shape

(1716960, 41)

In [ ]:
# dfOut.loc[dfOut['Zipcode']==10075,['time','Construction']].sum()
# df1 = pd.read_csv("/Users/graceli/Documents/Baruch MSBA/S3/BUS9430/Data/FinalData/Events_updated_v2.csv")
# df1.shape

In [ ]:
# dfOut.loc[(dfOut['Zipcode']==10009)&(dfOut['time']>='2017-04-09')&(dfOut['time']<'2017-04-10'),['Special Event']].sum()

In [ ]:
# df2 = pd.read_csv("/Users/graceli/Documents/Baruch MSBA/S3/BUS9430/Data/FinalData/Events_updated.csv")
# df2.shape